In [1]:
import matplotlib.pyplot as plt
from classiq import construct_combinatorial_optimization_model
from classiq.applications.combinatorial_optimization import OptimizerConfig, QAOAConfig
import numpy as np
from qiskit import QuantumCircuit as qc
from pyomo.environ import *
from pyomo.opt import SolverFactory
from classiq import show, synthesize
from classiq import execute
import pandas as pd
from classiq.applications.combinatorial_optimization import get_optimization_solution_from_pyo
from classiq import set_execution_preferences
from classiq.execution import ClassiqBackendPreferences, ExecutionPreferences
from classiq import write_qmod
import random as rd

In [2]:
c_test = [[1,0,0,0],[1,0,1,0],[0,-1,0,0],[0,0,0,-1],[0,0,1,1]]

def model_from_mat(p_mat=c_test):
    cost = 0
    n, m = len(p_mat[0]), len(p_mat)
    model = ConcreteModel()
    model.x = Var(range(n), domain=Binary)
    for i in range(m):
        clause_i = 1
        for j in range(n):
            if(p_mat[i][j] == 1):
                clause_i *= 1-model.x[j]
            if(p_mat[i][j] == -1):
                clause_i *= model.x[j]
        cost += clause_i
    model.obj = Objective(expr=cost, sense=minimize)
    return model

In [10]:
def solve_ksat_qaoa(p_mat=c_test,penalty_energy=2,alpha_cavar=0.9,max_iter=500,backend_name="aer_simulator"):
    model = model_from_mat(p_mat)
    qaoa_config = QAOAConfig(num_layers=1, penalty_energy=2)
    optimizer_config = OptimizerConfig(max_iteration=500, alpha_cvar=0.9)
    qmod = construct_combinatorial_optimization_model(
        pyo_model=model,
        qaoa_config=qaoa_config,
        optimizer_config=optimizer_config,
    )
    backend_preferences = ExecutionPreferences(
        backend_preferences=ClassiqBackendPreferences(backend_name="aer_simulator")
    )
    qmod = set_execution_preferences(qmod, backend_preferences)
    qprog = synthesize(qmod)
    res = execute(qprog).result()
    vqe_result = res[0].value
    solution = get_optimization_solution_from_pyo(
        model, vqe_result=vqe_result, penalty_energy=qaoa_config.penalty_energy
    )
    optimization_result = pd.DataFrame.from_records(solution)
    idx = optimization_result.cost.idxmin()
    sol = optimization_result.solution[idx]
    cost = optimization_result.cost[idx]
    prob = optimization_result.probability[idx]
    table = pd.DataFrame.from_records(solution)
    print(f"QAOA solution: {sol}\n QAOA probability: {prob}\n QAOA cost: {cost}")
    return sol, prob, cost, table

In [11]:
ctest_2 = [[1,0,0,0,0],[1,0,1,0,0],[0,-1,0,0,0],[0,0,0,-1,0],[0,0,1,1,0],[0,0,0,0,-1],[1,0,0,0,1]]
solve_ksat_qaoa(ctest_2)

QAOA solution: [1, 0, 1, 0, 0]
 QAOA probability: 0.35937499999999994
 QAOA cost: 0.0


In [5]:
def solve_ksat_class(matrix=c_test):
    model = ConcreteModel()
    n, m = len(matrix[0]), len(matrix)
    model.x = Var(range(n), domain=Binary)
    model.obj = Objective(expr=0, sense=minimize)
    model.constraints = ConstraintList()

    for i in range(m):
        clause_expr = 0
        for j in range(n):
            if matrix[i][j] == 1:
                clause_expr += model.x[j]  # Variable itself
            elif matrix[i][j] == -1:
                clause_expr += (1 - model.x[j])  # Negation of the variable
        # Add at least one true literal per clause
        model.constraints.add(clause_expr >= 1)
        solver = SolverFactory('glpk')
        solver.solve(model, tee=False)
    solarray = [int(model.x[i].value) for i in range(n)]
    print(f"Classical solution : {solarray}")
    return [int(model.x[i].value) for i in range(n)]

def solve_ksat(matrix=c_test):
    csol = solve_ksat_class(matrix)
    qsol, qprob, qcost, table = solve_ksat_mat(matrix)
    if(csol == qsol):
        print("Both solutions are equal!")
    return csol, qsol, qprob, qcost, table

In [12]:
def generate_satisfiable_ksat_matrix(k, m, p=0.5): # p is the probability of including a variable in a clause
    matrix = np.zeros((m, k), dtype=int)
    for i in range(m):
        for j in range(k):
            if rd.random() < p:  # Probability of including a variable in a clause
                # Assign either 1 or -1 (true or negated)
                matrix[i, j] = 1 if rd.random() < 0.5 else -1
    return matrix

In [63]:
ran_mat_test = generate_satisfiable_ksat_matrix(4, 4, p=0.5)


In [67]:
list(ran_mat_test)

[array([ 0,  0, -1,  0]),
 array([ 0,  0,  1, -1]),
 array([ 0, -1, -1, -1]),
 array([ 0,  0,  0, -1])]

In [65]:
solve_ksat_class(ran_mat_test)

    solver failure.
    solver failure.
    solver failure.
    solver failure.


TypeError: int() argument must be a string, a bytes-like object or a real number, not 'NoneType'